In [1]:
import gzip
from collections import defaultdict
def readGz(f):
    for l in gzip.open(f):
        yield eval(l)
print('reading data...')
training = list(readGz('train.json.gz'))
print('done')

reading data...
done


In [ ]:
#training = data
#validation = data[int(len(data)/2):]

In [2]:
user_iden = [d['userID'] for d in training]
user_iden = list(set(user_iden))
business_iden = [d['businessID'] for d in training]
business_iden = list(set(business_iden))
training_data = [(d['userID'],d['businessID'],d['rating']) for d in training]
u_b_rating = defaultdict(float)
user_business = defaultdict(list)
business_user = defaultdict(list)
for (u,b,r) in training_data:
    u_b_rating[(u,b)] = r
    user_business[u].append(b)
    business_user[b].append(u)
for u in user_business:
    user_business[u] = list(set(user_business[u]))
for b in business_user:
    business_user[b] = list(set(business_user[b]))
beta_user = defaultdict(float)
beta_business = defaultdict(float)
gamma_user = defaultdict(float)
gamma_business = defaultdict(float)




In [3]:
allRating = []
userRating = defaultdict(list)
businessRating = defaultdict(list)
for (u,b,r) in training_data:
    allRating.append(r)
    userRating[u].append(r)
    businessRating[b].append(r)
globalAverage = sum(allRating)/len(allRating)
userAverage = {}
for u in userRating:
    userAverage[u] = sum(userRating[u])/len(userRating[u])
businessAverage = {}
for b in businessRating:
    businessAverage[b] = sum(businessRating[b])/len(businessRating[b])

In [4]:
#initialize
from random import randint
lam = 4
alpha = randint(40,45)/10
for u in user_iden:
    beta_user[u] = randint(-20,20)/100
    gamma_user[u] = randint(-20,20)/100
for b in business_iden:
    beta_business[b] = randint(-20,20)/100
    gamma_business[b] = randint(-20,20)/100

In [5]:
#update
#(1) fix gamma_business, update gamma_user
for iteration in range(2):
    for u in user_iden:
        update_gamma_user = 0
        sum_gamma_b = 0
        for b in user_business[u]:
            update_gamma_user += ( u_b_rating[(u,b)]-alpha-beta_user[u]-beta_business[b] )*gamma_business[b]
            sum_gamma_b += gamma_business[b]**2 
        gamma_user[u] = update_gamma_user/( sum_gamma_b + lam)
    #(2) fix gamma_user , update gamma_business
    for b in business_iden:
        update_gamma_business = 0
        sum_gamma_u = 0
        for u in business_user[b]:
            update_gamma_business += ( u_b_rating[(u,b)]-alpha-beta_user[u]-beta_business[b] )*gamma_user[u]
            sum_gamma_u += gamma_user[u]**2
        gamma_business[b] = update_gamma_business/( sum_gamma_u + lam)
    #(3) fix gamma_user and gamma_business, update alpha
    update_alpha = 0
    for (u,b,r) in training_data:
        update_alpha += r - beta_user[u]-beta_business[b]-gamma_user[u]*gamma_business[b]
    alpha = update_alpha / len(training_data)
    #(4) fix gamma_business, gamma_user, alpha , update beta_user
    for u in user_iden:
        update_beta_user = 0
        for b in user_business[u]:
            update_beta_user += u_b_rating[(u,b)] - alpha-beta_business[b]-gamma_user[u]*gamma_business[b]
        beta_user[u] = update_beta_user/(len(user_business[u]) + lam)
    for b in business_iden:
        update_beta_business = 0
        for u in business_user[b]:
            update_beta_business += u_b_rating[(u,b)] - alpha-beta_user[u]-gamma_user[u]*gamma_business[b]
        beta_business[b] = update_beta_business/(len(business_user[b])+lam)
    
    

In [6]:
predictions = open('task2_rating.txt','w')
for l in open('pairs_Rating.txt'):
    if l.startswith('userID'):
        predictions.write(l)
        continue
    u,b = l.strip().split('-')
    if u in user_iden:
        if b in business_iden:
            rating = alpha+beta_user[u]+beta_business[b]+gamma_user[u]*gamma_business[b]
            predictions.write(u+'-'+b+','+str(rating)+'\n')
        else:
            rating = alpha+beta_user[u]
            predictions.write(u+'-'+b+','+str(rating)+'\n')
    else:
        if b in business_iden:
            rating = alpha+beta_business[b]
            predictions.write(u+'-'+b+','+str(rating)+'\n')
        else:
            rating = alpha
            predictions.write(u+'-'+b+','+str(rating)+'\n')
predictions.close()
            

In [ ]:
# find lambda = 4, iteration time =2 ,the best result

In [ ]:
#MSE of training data
MSE_training = 0
for (u,b,r) in training_data:
    MSE_training += (alpha+beta_user[u]+beta_business[b]+gamma_user[u]*gamma_business[b]-r)**2
MSE_training = MSE_training/len(training)
print('MSE for training_data is :',MSE_training)
#MSE of validation set
validation_data = [(d['userID'],d['businessID'],d['rating']) for d in validation]
MSE_validation = 0
for (u,b,r) in validation_data:
    if u in user_iden:
        if b in business_iden:
            MSE_validation += (alpha+beta_user[u]+beta_business[b]+gamma_user[u]*gamma_business[b]-userAverage[u])**2
        else:
            MSE_validation += (alpha+beta_user[u]-r)**2
    else:
        if b in business_iden:
            MSE_validation += (alpha+beta_user[u]+beta_business[b]+gamma_user[u]*gamma_business[b]-businessAverage[b])**2
        else:
            MSE_validation += (alpha-globalAverage)**2
MSE_validation = MSE_validation/len(validation_data)
print('MSE for validation_data :',MSE_validation)
    

In [ ]:
#MSE of training data
    MSE_training = 0
    for (u,b,r) in training_data:
        MSE_training += (alpha+beta_user[u]+beta_business[b]+gamma_user[u]*gamma_business[b]-r)**2
    MSE_training = MSE_training/len(training)
    print('MSE for training_data is :',MSE_training)
    #MSE of validation set
    validation_data = [(d['userID'],d['businessID'],d['rating']) for d in validation]
    MSE_validation = 0
    for (u,b,r) in validation_data:
        if u in user_iden:
            if b in business_iden:
                MSE_validation += (alpha+beta_user[u]+beta_business[b]+gamma_user[u]*gamma_business[b]-r)**2
            else:
                MSE_validation += (alpha+beta_user[u]-userAverage[u])**2
        else:
            if b in business_iden:
                MSE_validation += (alpha+beta_business[b]-businessAverage[b])**2
            else:
                MSE_validation += (alpha-globalAverage)**2
    MSE_validation = MSE_validation/len(validation_data)
    print('MSE for validation_data :',MSE_validation)
    